# Compute a lower bound for the new TAP-C

To compute a lower bound, we first find a dual solution, then use it to derive a lower bound as sensitivity analysis.

In [7]:
import numpy as np

In [12]:
import cvxpy as cvx

In [64]:
def lower_bound(f, h, t, Delta, route2od, u_old, u_new, nk):
    """ 
    function that compute a lower bound with updated capacity given the original solution of a TAP-C
    input:
    f(vector): primal solution of link flow
    h(vector): primal solution of route flow
    t(ventor): link travel time of f
    Delta(matrix)[r,a]: 1 if route r use link a; 0 otherwise. 
    route2od(vector): mapping the route index to the index of o-d pairs
    u_old(vector): original capacity of links
    u_new(vector): new capacity of links   
    """
    nr, na = Delta.shape # number of non-zero routes and number of links 
    c = Delta.dot(t)
    
    lambda_link = cvx.Variable(na)
    pi = cvx.Variable(nk)
    
    obj = cvx.Maximize((u_old - u_new) * lambda_link)
    
    constr_1 = [lambda_link[i]==0 for i in range(na) if f[i]!=u_old[i]]
    constr_2 = [lambda_link[i]>=0 for i in range(na)]
    constr_3 = [c[r]+Delta[r]*lambda_link == pi[route2od[r]] for r in range(nr) if h[r]>0]
    constr_4 = [c[r]+Delta[r]*lambda_link >= pi[route2od[r]] for r in range(nr)]
    
    constr = constr_1 + constr_2 + constr_3 + constr_4
    
    problem = cvx.Problem(obj, constr)
    bound = problem.solve()
    
    return bound, lambda_link.value, pi.value

In [65]:
f = np.array([1.0,1.1,0.1])
h = np.array([1.0, 0.1])
t = np.array([3.0, 2.1, 9.0])
Delta = np.array([[1,1,0], [0,1,1]])
route2od = [0,0]
u_old = np.array([1.0, 2.0, 3.0])
u_new = np.array([0.5, 0.9, 3.0])
nk = 1

In [66]:
lower_bound(f, h, t, Delta, route2od, u_old, u_new, nk)

(3.000000002275825,
 array([ 6.00000000e+00, -3.69194454e-11, -9.86739310e-09]),
 array([11.10000004]))